In [86]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
        
    return str_text

In [87]:
read_file('moby_dick_four_chapters.txt')

'Call me Ishmael.  Some years ago--never mind how long\nprecisely--having little or no money in my purse, and nothing\nparticular to interest me on shore, I thought I would sail about a\nlittle and see the watery part of the world.  It is a way I have of\ndriving off the spleen and regulating the circulation.  Whenever I\nfind myself growing grim about the mouth; whenever it is a damp,\ndrizzly November in my soul; whenever I find myself involuntarily\npausing before coffin warehouses, and bringing up the rear of every\nfuneral I meet; and especially whenever my hypos get such an upper\nhand of me, that it requires a strong moral principle to prevent me\nfrom deliberately stepping into the street, and methodically knocking\npeople\'s hats off--then, I account it high time to get to sea as soon\nas I can.  This is my substitute for pistol and ball.  With a\nphilosophical flourish Cato throws himself upon his sword; I quietly\ntake to the ship.  There is nothing surprising in this.  If t

In [88]:
import spacy

In [89]:
nlp = spacy.load('en')

In [90]:
nlp.max_length = 1198623

In [91]:
def seperate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n"-#$%&()--.*+-/:;<=>?[\\]^_`{|}~\t\n ']

In [92]:
d = read_file('moby_dick_four_chapters.txt')

In [93]:
tokens = seperate_punc(d)

In [94]:
len(tokens)

12360

In [95]:
#25 words ----> network predict #26

In [96]:
train_len = 25+1
text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

In [97]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse , and nothing particular to interest me'

In [98]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse , and nothing particular to interest me on'

In [99]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse , and nothing particular to interest me on shore'

In [100]:
from keras.preprocessing.text import Tokenizer


In [101]:
tokenizer = Tokenizer()

In [102]:
tokenizer.fit_on_texts(text_sequences)

In [103]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [104]:
# sequences[1]

In [105]:
tokenizer.index_word

{1: ',',
 2: 'the',
 3: 'a',
 4: 'and',
 5: 'of',
 6: 'i',
 7: 'to',
 8: 'in',
 9: 'it',
 10: 'that',
 11: 'he',
 12: 'his',
 13: 'was',
 14: 'but',
 15: 'me',
 16: 'with',
 17: 'as',
 18: 'you',
 19: 'this',
 20: 'at',
 21: 'is',
 22: 'all',
 23: 'for',
 24: 'my',
 25: 'on',
 26: 'be',
 27: "'s",
 28: 'not',
 29: 'from',
 30: 'there',
 31: 'one',
 32: 'up',
 33: 'what',
 34: 'him',
 35: 'so',
 36: 'bed',
 37: 'now',
 38: 'no',
 39: 'about',
 40: 'into',
 41: 'by',
 42: 'were',
 43: 'out',
 44: 'or',
 45: '!',
 46: 'harpooneer',
 47: 'had',
 48: 'then',
 49: 'have',
 50: 'an',
 51: 'upon',
 52: 'little',
 53: 'some',
 54: 'old',
 55: 'like',
 56: 'if',
 57: 'they',
 58: 'would',
 59: 'do',
 60: 'over',
 61: 'landlord',
 62: 'thought',
 63: 'room',
 64: 'when',
 65: 'could',
 66: 'here',
 67: 'head',
 68: "n't",
 69: 'night',
 70: 'such',
 71: 'which',
 72: 'man',
 73: 'did',
 74: 'sea',
 75: 'though',
 76: 'time',
 77: 'other',
 78: 'very',
 79: 'go',
 80: 'these',
 81: 'more',
 82: 'f

In [106]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

966 : call
15 : me
267 : ishmael
53 : some
265 : years
418 : ago
89 : never
224 : mind
131 : how
113 : long
964 : precisely
264 : having
52 : little
44 : or
38 : no
323 : money
8 : in
24 : my
557 : purse
1 : ,
4 : and
152 : nothing
263 : particular
7 : to
2706 : interest
15 : me


In [107]:
# tokenizer.word_counts

In [108]:
vocabulary_size = len(tokenizer.word_counts)

In [109]:
import numpy as np

In [110]:
sequences = np.array(sequences)

In [111]:
sequences

array([[ 966,   15,  267, ...,    7, 2706,   15],
       [  15,  267,   53, ..., 2706,   15,   25],
       [ 267,   53,  265, ...,   15,   25,  967],
       ...,
       [2705,    1,    4, ...,  266,   55,    3],
       [   1,    4,   11, ...,   55,    3, 2711],
       [   4,   11, 2707, ...,    3, 2711,   27]])

In [112]:
from keras.utils import to_categorical

In [113]:
X = sequences[:,:-1]

In [114]:
y = sequences[:,-1]

In [115]:
y = to_categorical(y,num_classes=vocabulary_size+1)

In [116]:
y.shape

(12334, 2712)

In [117]:
X.shape

(12334, 25)

In [118]:
from keras.models import Sequential

In [119]:
from keras.layers import Dense, LSTM, Embedding

In [120]:
def create_model (vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size,seq_len, input_length=seq_len))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    model.summary()
    return model

In [121]:
seq_len = X.shape[1]

In [122]:
model = create_model(vocabulary_size+1, seq_len)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 25, 25)            67800     
_________________________________________________________________
lstm_7 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_8 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_8 (Dense)              (None, 2712)              138312    
Total params: 244,062
Trainable params: 244,062
Non-trainable params: 0
_________________________________________________________________


In [123]:
from pickle import dump,load

In [124]:
len(X)
X.shape

(12334, 25)

In [125]:
len(y)
y.shape

(12334, 2712)

In [126]:
model.fit(X,y,batch_size=128, epochs=2, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
12334/12334 [==============================] - ETA: 4:28 - loss: 7.9054 - acc: 0.0000e+0 - ETA: 2:15 - loss: 7.9051 - acc: 0.0000e+0 - ETA: 1:31 - loss: 7.9046 - acc: 0.0000e+0 - ETA: 1:08 - loss: 7.9040 - acc: 0.0020    - ETA: 55s - loss: 7.9035 - acc: 0.004 - ETA: 46s - loss: 7.9029 - acc: 0.00 - ETA: 40s - loss: 7.9024 - acc: 0.00 - ETA: 35s - loss: 7.9018 - acc: 0.00 - ETA: 28s - loss: 7.8998 - acc: 0.01 - ETA: 25s - loss: 7.8989 - acc: 0.01 - ETA: 23s - loss: 7.8975 - acc: 0.01 - ETA: 22s - loss: 7.8959 - acc: 0.01 - ETA: 20s - loss: 7.8945 - acc: 0.01 - ETA: 19s - loss: 7.8926 - acc: 0.02 - ETA: 17s - loss: 7.8884 - acc: 0.02 - ETA: 16s - loss: 7.8845 - acc: 0.02 - ETA: 15s - loss: 7.8798 - acc: 0.02 - ETA: 14s - loss: 7.8737 - acc: 0.02 - ETA: 13s - loss: 7.8555 - acc: 0.02 - ETA: 12s - loss: 7.8473 - acc: 0.02 - ETA: 12s - loss: 7.8330 - acc: 0.02 - ETA: 11s - loss: 7.8155 - acc: 0.02 - ETA: 11s - loss: 7.7975 - acc: 0.0

In [127]:
model.save('my_mobydick_model.h5')

In [128]:
dump(tokenizer, open('my_simpletokenizer','wb'))

In [132]:
from keras.preprocessing.sequence import pad_sequences

In [145]:
def generate_text(model,tokenizer,seq_len,seed_text,num_get_words):
    output_text = []
    input_text = seed_text
    
    for i in range(num_get_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating = 'pre')
        pre_word_ind = model.predict_classes(pad_encoded,verbose=0)[0]
        pre_word = tokenizer.index_word[pre_word_ind]
        input_text += ' '+pre_word
        output_text.append(pre_word)
    return ' '.join(output_text)

In [136]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 ',',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me']

In [137]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [138]:
random_seed_text = text_sequences[random_pick]

In [139]:
random_seed_text

['real',
 'baby',
 'preserved',
 'in',
 'some',
 'similar',
 'manner',
 'but',
 'seeing',
 'that',
 'it',
 'was',
 'not',
 'at',
 'all',
 'limber',
 ',',
 'and',
 'that',
 'it',
 'glistened',
 'a',
 'good',
 'deal',
 'like',
 'polished']

In [140]:
seed_text= ' '.join(random_seed_text)

In [159]:
seed_text2 = 'i think that i need to stop wasting my time and focus on something that has a greater importance in life such as studies of '

In [148]:
generate_text(model, tokenizer,seq_len,seed_text=seed_text, num_get_words=25)

', , , , , , , , , , , , , , , , , , , , , , , , ,'

In [149]:
from keras.models import load_model

In [154]:
model = load_model('epochBIG.h5')

In [156]:
tokenizer = load(open('epochBIG', 'rb'))

In [162]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_get_words=25)

'ebony spanish piece of two pagan terrible beings at every water like a veritable professors complement of both junk has he tossed in the distracted'